# RAG 기반 QA 봇

In [47]:
from bs4 import BeautifulSoup, SoupStrainer
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain_community.document_loaders import WebBaseLoader 
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langchain.callbacks.base import BaseCallbackHandler

# SoupStrainer 이해

In [18]:
import requests

url = "https://news.naver.com/breakingnews/section/105/230"

client = requests.get(url)
html = client.text 
soup = BeautifulSoup(html, 'html.parser')
soup

<!DOCTYPE html>

<html data-useragent="python-requests/2.31.0" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport">
<meta content="IT/과학 : 네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/breakingnews/section/105/230" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png" property="og:image"/>
<meta content="모바일, 인터넷, SNS, 통신 등 IT/과학 분야 뉴스 제공" property="og:description"/>
<meta content="네이버" property="og:article:author"/>
<meta content="summary" name="twitter:card"/>
<meta content="IT/과학 : 네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"

In [19]:
import requests

url = "https://news.naver.com/breakingnews/section/105/230"

client = requests.get(url)
html = client.text 
only_read = SoupStrainer("ul", attrs={"class":"sa_list"})           # 필요한 부분만 긁어온다.
soup = BeautifulSoup(html, 'html.parser', parse_only=only_read)
soup

<ul class="sa_list">
<li class="sa_item _LAZY_LOADING_WRAP">
<div class="sa_item_inner">
<div class="sa_item_flex">
<div class="sa_thumb _LAZY_LOADING_ERROR_HIDE">
<div class="sa_thumb_inner">
<a class="sa_thumb_link" data-clk="alist" data-gdid="880000EA_000000000000000003201607" data-rank="1" href="https://n.news.naver.com/mnews/article/030/0003201607">
<img alt="포스코퓨처엠, 혼다와 加 양극재 합작사 설립" class="_LAZY_LOADING _LAZY_LOADING_INIT_HIDE" data-src="https://mimgnews.pstatic.net/image/origin/030/2024/04/26/3201607.jpg?type=nf220_150" height="75" onerror="this.outerHTML='&lt;span class=&quot;noimage&quot;&gt;&lt;/span&gt;'" style="display: none;" width="110"/>
</a>
</div>
</div>
<div class="sa_text">
<a class="sa_text_title" data-clk="alist" data-gdid="880000EA_000000000000000003201607" data-rank="1" href="https://n.news.naver.com/mnews/article/030/0003201607">
<strong class="sa_text_strong">포스코퓨처엠, 혼다와 加 양극재 합작사 설립</strong>
</a>
<div class="sa_text_lede">포스코퓨처엠이 일본 혼다와 캐나다에 양극재 합작사를 설립한다. 양사

# 다시 시작

In [65]:
# 웹 불러오기
url = "https://n.news.naver.com/mnews/ranking/article/022/0003927670?ntype=RANKING"

strainer = SoupStrainer("div", attrs={"class":"newsct_article _article_body"})           # 필요한 부분만 긁어온다.
loader = WebBaseLoader(
    web_paths = (url, ),
    bs_kwargs=dict(
        parse_only=strainer
    )
)

docs = loader.load()
print(docs)

[Document(page_content='\n\n피해자母 “출소 후 재범하면 누가 책임지냐” \n\n\n\n   살인사건 가해자 류모씨(왼쪽부터)와 피해자 정혜주씨, 차경미씨. 연합뉴스    결혼을 약속한 여자친구를 흉기로 200회 가까이 찔러 잔혹하게 살해한 20대의 죗값이 징역 23년으로 확정됐다.  \xa0  25일 법조계에 따르면 지난 17일 살인 혐의로 징역 23년을 선고받은 류모(28)씨는 상고 마감 기한인 24일까지 서울고법 춘천재판부에 상고장을 내지 않았다.  \xa0  대법원은 사실관계를 다루는 1·2심과 달리 법률심으로 하게 돼 있으나 형사소송법상 사형, 무기 또는 10년 이상의 징역이나 금고가 선고된 사건의 경우 형의 양정이 심히 부당하다고 인정할 현저한 사유가 있는 때에는 상고할 수 있다.  \xa0  징역 25년을 구형했던 검찰도 상고하지 않았다.  \xa0  ◆“여자친구 난도질했거든요” 112에 스스로 신고…재판서 “정신지체냐는 말 듣고 격분해 범행” 진술  연합뉴스에 따르면 류씨는 지난해 7월 24일 낮 12시 47분께 영월군 영월읍 덕포리 한 아파트에서 정혜주(사망 당시 24세)씨를 흉기로 191회 찔러 살해한 혐의로 재판에 넘겨졌다.  \xa0  그는 6분 뒤 "제가 여자친구를 죽였어요", "여자친구를 난도질했거든요"라며 112에 스스로 신고했다.  \xa0  류씨는 수사기관에서 옆집과 층간소음 문제로 갈등을 겪어 스트레스를 받아 범행했다거나 결혼을 앞두고 경제적으로 곤궁한 상황으로 인해 스트레스가 쌓이던 중 문득 \'여자친구를 살해하면 고통에서 해방될 수 있겠다\'는 생각에 사로잡혀 순간적으로 실행에 옮겼다고 주장했다.  \xa0  그러나 1심 재판에서는 "피해자로부터 \'정신지체냐\'라는 말을 듣고 격분해 범행했다"고 진술을 바꿨다.  \xa0  1심은 류씨가 층간소음 문제와 경제적 곤궁 등으로 인한 스트레스로부터 해방되기 위해 살해했다는 점이 선뜻 이해되지 않고, 이례적인 범행동기를 가질 만한 정신질환도 없었던 점을 근거로

In [67]:
# Split (chunk_size 청크의 길이, chunk overlab 분할할때 중첩 허용)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

splits = text_splitter.split_documents(docs)
print(f"분할된 문서의 수: {len(splits)}")
for chunk in splits:
    print(chunk.page_content)
    print("="*100)

분할된 문서의 수: 5
피해자母 “출소 후 재범하면 누가 책임지냐”
살인사건 가해자 류모씨(왼쪽부터)와 피해자 정혜주씨, 차경미씨. 연합뉴스    결혼을 약속한 여자친구를 흉기로 200회 가까이 찔러 잔혹하게 살해한 20대의 죗값이 징역 23년으로 확정됐다.     25일 법조계에 따르면 지난 17일 살인 혐의로 징역 23년을 선고받은 류모(28)씨는 상고 마감 기한인 24일까지 서울고법 춘천재판부에 상고장을 내지 않았다.     대법원은 사실관계를 다루는 1·2심과 달리 법률심으로 하게 돼 있으나 형사소송법상 사형, 무기 또는 10년 이상의 징역이나 금고가 선고된 사건의 경우 형의 양정이 심히 부당하다고 인정할 현저한 사유가 있는 때에는 상고할 수 있다.     징역 25년을 구형했던 검찰도 상고하지 않았다.     ◆“여자친구 난도질했거든요” 112에 스스로 신고…재판서 “정신지체냐는 말 듣고 격분해 범행” 진술  연합뉴스에 따르면 류씨는 지난해 7월 24일 낮 12시 47분께 영월군 영월읍 덕포리 한 아파트에서 정혜주(사망 당시
7월 24일 낮 12시 47분께 영월군 영월읍 덕포리 한 아파트에서 정혜주(사망 당시 24세)씨를 흉기로 191회 찔러 살해한 혐의로 재판에 넘겨졌다.     그는 6분 뒤 "제가 여자친구를 죽였어요", "여자친구를 난도질했거든요"라며 112에 스스로 신고했다.     류씨는 수사기관에서 옆집과 층간소음 문제로 갈등을 겪어 스트레스를 받아 범행했다거나 결혼을 앞두고 경제적으로 곤궁한 상황으로 인해 스트레스가 쌓이던 중 문득 '여자친구를 살해하면 고통에서 해방될 수 있겠다'는 생각에 사로잡혀 순간적으로 실행에 옮겼다고 주장했다.     그러나 1심 재판에서는 "피해자로부터 '정신지체냐'라는 말을 듣고 격분해 범행했다"고 진술을 바꿨다.     1심은 류씨가 층간소음 문제와 경제적 곤궁 등으로 인한 스트레스로부터 해방되기 위해 살해했다는 점이 선뜻 이해되지 않고, 이례적인 범행동기를 가질 만한 정신질환도 없었던 점을 근거로 피해자로부

In [68]:
# vectorstore 
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# retriever 
retriever = vectorstore.as_retriever()

In [69]:
# prompt
prompt = hub.pull("rlm/rag-prompt")
print(prompt.messages[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [70]:
class CustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    streaming=True,
    callbacks=[CustomHandler()]
)

output_parser = StrOutputParser()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm 
    | output_parser
)

In [72]:
_ = rag_chain.invoke(
    "범죄를 저지른 원인이 뭔가요?"
)

스트레스와 경제적 곤궁으로 인해 범행을 저지른 것으로 주장되며, 피해자로부터 '정신지체냐'라는 말을 듣고 우발적으로 범행했다고 진술했습니다. 재판부는 범행 동기와 관련해 피고인이 피해자를 살해한 동기를 임의로 단정하기 어렵다고 판단했습니다.

In [75]:
_ = rag_chain.invoke(
    "어떤 사건인지 설명해주세요"
)

정혜주(사망 당시 24세)씨를 흉기로 191회 찔러 살해한 사건에 대한 범행 동기와 관련하여 피고인은 스트레스와 경제적 곤궁으로 인해 살해를 실행했다고 주장했으나, 재판부는 이를 이해하기 어렵다고 판단했습니다. 원심은 피해자로부터 '정신지체냐'라는 말을 듣고 우발적으로 범행했다는 결론을 내렸습니다.

In [73]:
_ = rag_chain.invoke(
    "재판 결과가 뭔가요?"
)

원심은 피고인을 23년의 징역형에 처해하고, 위치추적 전자장치 부착 명령은 기각했습니다. 피해자의 어머니는 피고인의 재범 가능성을 우려했으며, 항소심 재판부는 범행 동기를 임의로 단정하기 어렵다고 판단했습니다.

# Q&A 봇 두번째 방법

In [ ]:
# 웹 불러오기
url = "https://www.samsungsds.com/kr/insights/ai-in-telecom-and-finance.html"

strainer = SoupStrainer("div", attrs={"class":"newsct_article _article_body"})           # 필요한 부분만 긁어온다.
loader = WebBaseLoader(
    web_paths = (url, ),
    bs_kwargs=dict(
        parse_only=strainer
    )
)

docs = loader.load()
print(docs)